In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
print(pd.__version__)
%matplotlib inline




# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

0.23.2
1.1.3


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [36]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [37]:
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
complete_data=df_train.dropna()
# 排除獨特資料
complete_data = complete_data.drop(['Name','Ticket','PassengerId'],axis = 1)
# display(complete_data)
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 889
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       183 non-null    int64  
 1   Pclass         183 non-null    int64  
 2   Sex            183 non-null    object 
 3   Age            183 non-null    float64
 4   SibSp          183 non-null    int64  
 5   Parch          183 non-null    int64  
 6   Fare           183 non-null    float64
 7   Cabin          183 non-null    object 
 8   Embarked       183 non-null    object 
 9   Survived_cate  183 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 15.7+ KB


In [38]:
num_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [39]:
cat_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

4 category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']



In [79]:
x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y=complete_data['Survived']
# display(x)
# display(y)
pd.set_option("display.max_rows", 1000)
x['Embarked']


1      C
3      S
6      S
10     S
11     S
21     S
23     S
27     S
52     C
54     C
62     S
66     S
75     S
88     S
92     S
96     C
97     C
102    S
110    S
118    C
123    S
124    S
136    S
137    S
139    C
148    S
151    S
170    S
174    C
177    C
183    S
193    S
194    C
195    C
205    S
209    C
215    C
218    C
224    S
230    S
245    Q
248    S
251    S
252    S
257    S
262    S
263    S
268    S
269    S
273    C
275    S
291    C
292    C
297    S
299    C
305    S
307    C
309    C
310    C
311    C
318    S
319    C
325    C
327    S
329    C
331    S
332    S
336    S
337    C
339    S
340    S
341    S
345    S
356    S
366    C
369    C
370    C
377    C
390    S
393    C
394    S
412    Q
429    S
430    S
434    S
435    S
438    S
445    S
449    S
452    C
453    C
456    S
460    S
462    S
473    C
484    C
486    S
487    C
492    S
496    C
498    S
504    S
505    C
512    S
515    S
516    S
520    S
523    C
536    S
539    C
540    S
5

In [84]:
# 先將離散資料轉換成數值，['Sex', 'Embarked']
sex_mapping = {'male': 1,'female': 0}
complete_data['s1'] = complete_data['Sex'].map(sex_mapping)
embarked_mapping = {'C': 0,'S': 1,'Q':2}
complete_data['e1'] = complete_data['Embarked'].map(embarked_mapping)
x = complete_data[['s1','e1','Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

In [85]:
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x, y)
# True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

[ True False False False  True False False]
[1 3 2 5 1 4 6]
['s1', 'SibSp']
